####Mount Azure DataLake using service principal
1. Get client_id,tenant_id and client_secret from key vault
1. Set Spark config with App/client-id, Directory/Tenant Id & Secret
1. Call file system utilities to mount the sotrage.
1. Explore otherfile system utilities related to mount(all mounts, unmounts)

In [0]:
def mount_adls(storage_account_name,container_name):
    #Get secret from key vault
    client_id=dbutils.secrets.get(scope="formula1-scope", key="formula15-app-client-id")
    tenant_id =dbutils.secrets.get(scope="formula1-scope", key="formula15-app-tenant-d")
    client_secret=dbutils.secrets.get(scope="formula1-scope", key="formula15-app-client-secret")

    #Set spark confinguration
    #storage_account="formuladl15"
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret":client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

#checking if mount is exiting or not!!
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
       dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    # mounting demo container
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)

    #display all list of mounted storage
    display(dbutils.fs.mounts())

In [0]:
#mounting silver layer
mount_adls('formuladl15','silver')
#mounting bronze layer
mount_adls('formuladl15','bronze')
#mounting gold layer
mount_adls('formuladl15','gold')

In [0]:
mount_adls('formuladl15','raw')


In [0]:
mount_adls('formuladl15','processed')

In [0]:
mount_adls('formuladl15','process')


In [0]:
mount_adls('formuladl15','presentation')

In [0]:
display(dbutils.fs.ls("/mnt/formuladl15/silver/2021-03-21/circuits.csv"))


In [0]:
# display(spark.read.options(header=True).csv("/mnt/formuladl15/silver/2021-03-21/circuits.csv"))